<a href="https://colab.research.google.com/github/Mengfan-Wei/colab/blob/main/chapter_linear-networks/linear-regression-concise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/d2l-ai/d2l-zh@release  # installing d2l


In [3]:
!pip install matplotlib_inline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 线性回归的简洁实现

## 生成数据集

In [9]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

In [5]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

## 读取数据集

我们可以[**调用框架中现有的API来读取数据**]。
我们将`features`和`labels`作为API的参数传递，并通过数据迭代器指定`batch_size`。
此外，布尔值`is_train`表示是否希望数据迭代器对象在每个迭代周期内打乱数据。


In [6]:
def load_array(data_arrays, batch_size, is_train=True): 
    """构造一个PyTorch数据迭代器
    is_train表示是否希望数据迭代器对象在每个迭代周期内打乱数据。"""
    dataset = data.TensorDataset(*data_arrays)  # 构建pytorch的dataset
    return data.DataLoader(dataset, batch_size, shuffle=is_train)   # 每次随机抽取batch_size的数据

In [7]:
batch_size = 10
data_iter = load_array((features, labels), batch_size)

使用`data_iter`的方式与我们在 :numref:`sec_linear_scratch`中使用`data_iter`函数的方式相同。为了验证是否正常工作，让我们读取并打印第一个小批量样本。
与 :numref:`sec_linear_scratch`不同，这里我们使用`iter`构造Python迭代器，并使用`next`从迭代器中获取第一项。


In [8]:
next(iter(data_iter))

[tensor([[ 0.4231,  0.4992],
         [ 2.1525,  0.4682],
         [-1.1366,  0.9799],
         [ 0.6892,  1.0206],
         [ 1.3306,  0.3035],
         [ 0.3024, -0.3223],
         [ 0.2218,  0.8722],
         [ 0.1903,  1.3367],
         [-1.7482,  2.0239],
         [-0.8273, -0.0885]]), tensor([[ 3.3539],
         [ 6.9151],
         [-1.4025],
         [ 2.1106],
         [ 5.8344],
         [ 5.9110],
         [ 1.6773],
         [ 0.0452],
         [-6.1773],
         [ 2.8588]])]

## 定义模型
在PyTorch中，全连接层在`Linear`类中定义


。
值得注意的是，我们将两个参数传递到`nn.Linear`中。
第一个指定输入特征形状，即2，第二个指定输出特征形状，输出特征形状为单个标量，因此为1。


In [10]:
# nn是神经网络的缩写
from torch import nn

net = nn.Sequential(nn.Linear(2, 1)) # 输入特征形状 - 2，输出特征形状为 - 1 （标量）

## **初始化模型参数**

我们指定每个权重参数应该从均值为0、标准差为0.01的正态分布中随机采样，
偏置参数将初始化为零。

In [11]:
# net[0]选择网络中的第一个图层， weight.data和bias.data方法访问参数，
# normal_和fill_方法来重写参数值
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

## 定义损失函数


In [12]:
loss = nn.MSELoss()

## 定义优化算法


小批量随机梯度下降算法是一种优化神经网络的标准工具，
PyTorch在`optim`模块中实现了该算法的许多变种。
当我们(**实例化一个`SGD`实例**)时，我们要指定优化的参数
（可通过`net.parameters()`从我们的模型中获得）以及优化算法所需的超参数字典。
小批量随机梯度下降只需要设置`lr`值，这里设置为0.03。


In [13]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

## 训练

在每个迭代周期里，将完整遍历一次训练集，
不停地从中获取一个小批量的输入和相应的标签。
对于每一个小批量，会进行以下步骤:

* 通过调用`net(X)`生成预测并计算损失`l`（前向传播）。
* 通过进行反向传播来计算梯度。
* 通过调用优化器来更新模型参数。


In [14]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad() # 置零梯度后进行反向传播
        l.backward()    
        trainer.step()  # step函数进行模型更新
    l = loss(net(features), labels)     # 计算每个迭代周期epoch后的损失
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000258
epoch 2, loss 0.000103
epoch 3, loss 0.000103


下面我们[**比较生成数据集的真实参数和通过有限数据训练获得的模型参数**]。
要访问参数，我们首先从`net`访问所需的层，然后读取该层的权重和偏置。
正如在从零开始实现中一样，我们估计得到的参数与生成数据的真实参数非常接近。


In [ ]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([-0.0008,  0.0006])
b的估计误差： tensor([0.0013])


## 小结


* 我们可以使用PyTorch的高级API更简洁地实现模型。
* 在PyTorch中，`data`模块提供了数据处理工具，`nn`模块定义了大量的神经网络层和常见损失函数。
* 我们可以通过`_`结尾的方法将参数替换，从而初始化参数。


## 练习

1. 如果将小批量的总损失替换为小批量损失的平均值，你需要如何更改学习率？
1. 查看深度学习框架文档，它们提供了哪些损失函数和初始化方法？用Huber损失代替原损失，即
    $$l(y,y') = \begin{cases}|y-y'| -\frac{\sigma}{2} & \text{ if } |y-y'| > \sigma \\ \frac{1}{2 \sigma} (y-y')^2 & \text{ 其它情况}\end{cases}$$
1. 你如何访问线性回归的梯度？


[Discussions](https://discuss.d2l.ai/t/1781)
